In [1]:
import os
import json
import numpy as np
import pandas as pd
import shutil

In [2]:
all_json = "/opt/ml/detection/object-detection-level2-cv-04/dataset/train.json"

In [3]:
with open(all_json, "r", encoding="utf8") as outfile:
    json_data = json.load(outfile)

images = json_data["images"]
categories = json_data["categories"]
annotations = json_data["annotations"]

In [4]:
for annotation in annotations:
    image_id = annotation["image_id"]
    category_id = annotation["category_id"]
    bbox = annotation["bbox"]

    if 'bbox' in images[image_id]:
        images[image_id]['bbox'].append(bbox)
        images[image_id]['bbox_category'].append(category_id)
    else:
        images[image_id]['bbox'] = [bbox]
        images[image_id]['bbox_category'] = [category_id]

In [6]:
# x, y, w, h, to x_centor, y_centor, w, h(normalized)
def convert(box):
    dw = 1./1024
    dh = 1./1024
    x, y, w, h = box
    x_c = (x + x + w) / 2.0
    y_c = (y + y + h) / 2.0
    x = x_c*dw
    w = w*dw
    y = y_c*dh
    h = h*dh
    return (x, y, w, h)

In [7]:
# image 파일 복사 & image 라벨 txt 만들기
def make_yolo_dataset(json_dir, image_dir, label_dir):
    with open(json_dir, "r", encoding="utf8") as outfile:
        json_data = json.load(outfile)
    yolo_images = json_data["images"]

    for yolo_image in yolo_images:
        id = yolo_image['id']
        img = images[id]
        string = ''
        for bbox, cate in zip(img['bbox'], img['bbox_category']):
            x, y, w, h = convert(bbox)
            string += f"{cate} {x} {y} {w} {h}\n"

        with open(os.path.join(label_dir,f"{img['id']:04}.txt"), 'w') as f:
            f.write(string.strip())
    
        shutil.copyfile(f"/opt/ml/detection/object-detection-level2-cv-04/dataset/{img['file_name']}", os.path.join(image_dir,f"{img['id']:04}.jpg"))

In [8]:
# make_directory
def make_directory(paths):
    for path in paths:
        os.makedirs(path, exist_ok=True)

In [9]:
# 경로 변경해서 실행
train_json = "/opt/ml/detection/object-detection-level2-cv-04/dataset/train.json"
train_image_path = '/opt/ml/detection/object-detection-level2-cv-04/yolov5/data/images/train'
train_label_path = '/opt/ml/detection/object-detection-level2-cv-04/yolov5/data/labels/train'
train_path = [train_image_path, train_label_path]

make_directory(train_path)
make_yolo_dataset(train_json, *train_path)

In [10]:
# 경로 변경해서 실행
valid_json = "/opt/ml/detection/object-detection-level2-cv-04/dataset/train.json"
valid_image_path = '/opt/ml/detection/object-detection-level2-cv-04/yolov5/data/images/valid'
valid_label_path = '/opt/ml/detection/object-detection-level2-cv-04/yolov5/data/labels/valid'
valid_path = [valid_image_path, valid_label_path]

make_directory(valid_path)
make_yolo_dataset(valid_json, *valid_path)